In [1]:
import plotly.express as px
import re
import plotly.io as pio
# pio.renderers.default = "browser"
pio.renderers.default = "notebook_connected"
from collections import OrderedDict
import argparse
from scipy.stats import gmean
import numpy as np
import plotly.graph_objects as go
import plotly.figure_factory as ff
import os
import sys
sys.path.append('..')
from util.spec2017_util import (
    bcolors,
    create_folder,
    yaml_load,
    yaml_overwrite,
)
from glob import glob
from scipy.stats import gmean

In [2]:
import pandas as pd
import numpy as np
pd.options.plotting.backend = "plotly"
if not os.path.exists("img"):
    os.mkdir("img")
w=3.3115
h=1.3

In [3]:
benchname = ['micro']
print(benchname)
snapshot_out = "out/"

['micro']


search all micro benchmark to find what to show in plots

In [4]:
bench_arr = []
bench_dict = dict()
for name in benchname:
    # print(name)
    files = glob(os.path.join(snapshot_out, f"{name}.*"))
    numf = len(files)
    # print(f"spec {name} contains {numf} outputs.")
    if numf == 0: continue
    # 2 comp schemes, 3 xor scheme, 2 constraints, 2 inter intra
    if (numf == 24): 
        bench_arr.append(name)

In [5]:
gem5_dir = "/home/zhewen/repo/gem5-dev/gem5"
num_ss = []
num_sample = 1
bench_num_arr = []
bench_cat_arr = []
for name in benchname:
    if name in bench_arr:
        num_ss.append(num_sample)
        bench_num_arr.append(-1)
        # print(val)
        bench_cat_arr.append('int')
print(num_ss)
print(bench_cat_arr)

[1]
['int']


initialize overall dataframe (dict_panda) and stat dataframe (bench_df)

In [6]:
dict_panda = {
    'compression': [],
    'xor': [],
    'config': [], # unconstratined, constrained
    'slice': [], # intra only, both inter and intra
    'bench': [], # a nested dataframe
    'bench_norm': [], # a nested dataframe (normalized)
}
df = pd.DataFrame(dict_panda)
display(df)

bench_df = pd.DataFrame(
    index=bench_arr, 
    data={
        'ss_theoretical':num_ss, 
        'bench_num':bench_num_arr,
        'category': bench_cat_arr,
    }
)
display(bench_df)

,compression,xor,config,slice,bench,bench_norm


,ss_theoretical,bench_num,category
micro,1,-1,int


 construct dataframe and fill

In [7]:
xor_schemes = ["none", "rand", "ideal"]
comp_schemes = ['bdi', 'dedup']
display_name_arr = ["bdi_none", "bdi_rand", "bdi_ideal", 
        "dedup_none", "dedup_rand", "dedup_ideal"]


bar_set_str_arr = ['_both', '']
config_str_arr = ['', 'uc']
for i in range(len(bar_set_str_arr)):
    bar_set_str = bar_set_str_arr[i]
    for j in range(len(config_str_arr)):
        config_str = config_str_arr[j]
        for comp in comp_schemes:
            for xor in xor_schemes:
                comb = f'{comp}{config_str}_{xor}'
                cr_arr = []
                cr_geo_arr = []
                ss_arr = []
                import copy
                local_bench_df = copy.deepcopy(bench_df)
                for name,ss in zip(bench_arr, num_ss):
                    f = open(snapshot_out + f"{name}.{comb}{bar_set_str}", "r")
                    str_ = f.readline()
                    if len(str_.split(','))==3:
                        cr = float(str_.split(',')[0])
                        ss = float(str_.split(',')[2])
                        cr_geo = float(str_.split(',')[1])
                        cr_arr.append(cr)
                        ss_arr.append(ss)
                        cr_geo_arr.append(cr_geo)
                    else: assert False
                    f.close()
                    
                # fill dataframe
                local_bench_df = local_bench_df.assign(cr_ari=cr_arr)
                local_bench_df = local_bench_df.assign(cr_geo=cr_geo_arr)
                local_bench_df = local_bench_df.assign(ss_actual=ss_arr)                
                local_bench_df_num = local_bench_df.select_dtypes(include='number')
                gm_arr = gmean(local_bench_df_num.iloc[:,:],axis=0)
                local_bench_df.loc['geomean'] = np.NaN
                local_bench_df.loc['geomean',local_bench_df_num.columns.tolist()] = gm_arr

                # copy dataframe
                local_bench_df_norm = copy.deepcopy(local_bench_df)

                config = 'constrained'
                slice_ = 'intra-only'
                if 'uc' in comb: config = 'unconstrained'
                if 'both' in bar_set_str: slice_ = 'both-inter-intra'
                df.loc[len(df.index)] = [comp, xor, config, slice_, local_bench_df, local_bench_df_norm] 

        

/home/zhewen/.local/lib/python3.8/site-packages/scipy/stats/_stats_py.py:310: RuntimeWarning:

invalid value encountered in log

/usr/local/lib/python3.8/dist-packages/pandas/core/dtypes/cast.py:949: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.

/home/zhewen/.local/lib/python3.8/site-packages/scipy/stats/_stats_py.py:310: RuntimeWarning:

invalid value encountered in log

/home/zhewen/.local/lib/python3.8/site-packages/scipy/stats/_stats_py.py:310: RuntimeWarning:

invalid value encountered in log

/home/zhewen/.local/lib/python3.8/site-packages/scipy/stats/_stats_py.py:310: RuntimeWarning:

invalid value encountered in log

/home/zhewen/.local/lib/python3.8/site-packages/scipy/stats/_stats_py.py:310: RuntimeWarning:

invalid value encountered in log

/home/zhewen/.lo

In [8]:
# denomerator config
xor_norm = 'none'
# comp_norm = 'bdi'
xor_schemes = ["none", "rand", "ideal"]
comp_schemes = ['bdi', 'dedup']
bar_set_str_arr = ['_both', '']
config_str_arr = ['', 'uc']

for i in range(len(bar_set_str_arr)):
    bar_set_str = bar_set_str_arr[i]
    for j in range(len(config_str_arr)):
        config_str = config_str_arr[j]
        for comp in comp_schemes:
            for xor in xor_schemes:
                config = 'constrained'
                slice_ = 'intra-only'
                if 'uc' in config_str: config = 'unconstrained'
                if 'both' in bar_set_str: slice_ = 'both-inter-intra'
                # print(xor, comp, config, slice_)
                # normalize dataframe
                base_overalldf = df.query(f"config == '{config}' and \
                    slice == '{slice_}' and \
                    xor == '{xor_norm}' and \
                    compression == '{comp}'")
                # display(base_overalldf)
                base_df = base_overalldf.bench.values[0]
                base_df_num = base_df.loc[:, ['cr_ari', 'cr_geo']]
                
                this_overalldf = df.query(f"config == '{config}' and \
                    slice == '{slice_}' and \
                    xor == '{xor}' and \
                    compression == '{comp}'")
                this_df = this_overalldf.bench_norm.values[0]
                this_df_num = this_df.loc[:, ['cr_ari', 'cr_geo']]
                this_df[this_df_num.columns] = this_df_num / base_df_num
                # display(this_df)
                ind = this_overalldf.index.values[0]
                df.at[ind,'bench_norm'] = this_df

summary of all data

In [9]:
display(df)

,compression,xor,config,slice,bench,bench_norm
0,bdi,none,constrained,both-inter-intra,ss_theoretical bench_num category ...,ss_theoretical bench_num category c...
1,bdi,rand,constrained,both-inter-intra,ss_theoretical bench_num category ...,ss_theoretical bench_num category ...
2,bdi,ideal,constrained,both-inter-intra,ss_theoretical bench_num category ...,ss_theoretical bench_num category ...
3,dedup,none,constrained,both-inter-intra,ss_theoretical bench_num category ...,ss_theoretical bench_num category c...
4,dedup,rand,constrained,both-inter-intra,ss_theoretical bench_num category ...,ss_theoretical bench_num category c...
5,dedup,ideal,constrained,both-inter-intra,ss_theoretical bench_num category ...,ss_theoretical bench_num category c...
6,bdi,none,unconstrained,both-inter-intra,ss_theoretical bench_num category ...,ss_theoretical bench_num category c...
7,bdi,rand,unconstrained,both-inter-intra,ss_theoretical bench_num category ...,ss_theoretical bench_num category ...
8,bdi,ideal,unconstrained,both-inter-intra,ss_theoretical bench_num category ...,ss_theoretical bench_num category ...
9,dedup,none,unconstrained,both-inter-intra,ss_theoretical bench_num category ...,ss_theoretical bench_num category c...


# constrained profiling
### L2 block can only xor with a block existing in L1

narrow down - constrained

In [10]:
df_c_intra = df.query("config == 'constrained' and slice == 'intra-only'")
display(df_c_intra)

df_c_both = df.query("config == 'constrained' and slice == 'both-inter-intra'")
display(df_c_both)


,compression,xor,config,slice,bench,bench_norm
12,bdi,none,constrained,intra-only,ss_theoretical bench_num category ...,ss_theoretical bench_num category c...
13,bdi,rand,constrained,intra-only,ss_theoretical bench_num category ...,ss_theoretical bench_num category ...
14,bdi,ideal,constrained,intra-only,ss_theoretical bench_num category ...,ss_theoretical bench_num category ...
15,dedup,none,constrained,intra-only,ss_theoretical bench_num category ...,ss_theoretical bench_num category c...
16,dedup,rand,constrained,intra-only,ss_theoretical bench_num category ...,ss_theoretical bench_num category c...
17,dedup,ideal,constrained,intra-only,ss_theoretical bench_num category ...,ss_theoretical bench_num category c...


,compression,xor,config,slice,bench,bench_norm
0,bdi,none,constrained,both-inter-intra,ss_theoretical bench_num category ...,ss_theoretical bench_num category c...
1,bdi,rand,constrained,both-inter-intra,ss_theoretical bench_num category ...,ss_theoretical bench_num category ...
2,bdi,ideal,constrained,both-inter-intra,ss_theoretical bench_num category ...,ss_theoretical bench_num category ...
3,dedup,none,constrained,both-inter-intra,ss_theoretical bench_num category ...,ss_theoretical bench_num category c...
4,dedup,rand,constrained,both-inter-intra,ss_theoretical bench_num category ...,ss_theoretical bench_num category c...
5,dedup,ideal,constrained,both-inter-intra,ss_theoretical bench_num category ...,ss_theoretical bench_num category c...


### Abs

In [11]:
fig = go.Figure()
fig.update_layout(
    # template='simple_white',
    title=f"XOR boost snapshot profiling (constrained,geomean)\
        <br><sup>BDI and dedup</sup>",
        yaxis_title="compression ratio",
        xaxis_title=None,
    font=dict(
        family="Courier New, monospace",
        size=18,
    ),
    legend=dict(
        orientation="h",
        itemwidth=30,
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5,
    ),
    barmode='group',
)
# fig.update_yaxes(range = [0,4])

i = 0
colors = [px.colors.sequential.Purp,px.colors.sequential.Tealgrn]
hatch = ['', '-']
for comp in df_c_intra.compression.unique():
    for xor in df_c_intra.xor.unique():
        q_overall = df_c_intra.loc[(df_c_intra['compression']==comp) & (df_c_intra['xor']==xor)]
        q_overall_both = df_c_both.loc[(df_c_both['compression']==comp) & (df_c_both['xor']==xor)]
        q = q_overall['bench'].values[0]
        q_both = q_overall_both['bench'].values[0]
        color = colors[int(q_overall.compression=='bdi')][i]
        color_both = colors[int(q_overall_both.compression=='bdi')][i]

        fig.add_trace(
            go.Bar(
                x=q_both.index, 
                y=q_both.cr_geo, 
                name=f'{comp}_{xor} both', 
                opacity=0.5,
                offsetgroup=i,
                marker_color=color,
                showlegend=False,
                marker_pattern_shape=[hatch[x]for x in q_both.category=='float']
            ),
        )

        fig.add_trace(
            go.Bar(
                x=q.index, 
                y=q.cr_geo, 
                name=f'{comp}_{xor}', 
                opacity=1,
                offsetgroup=i,
                marker_color=color_both,
                marker_pattern_shape=[hatch[x]for x in q.category=='float']
            ),
        )

        i+=1

fig.show()

# write static image
dpi = 300
fig.update_traces(marker_line_width = 0,
                  selector=dict(type="bar"),
                  marker_pattern_shape=None,)

fig.update_layout(
    template='plotly_white',
    width=2*w*dpi,# double col
    height=h*dpi,
    paper_bgcolor='rgba(0,0,0,0)',
    title=None,
    xaxis_title=None,
    yaxis_title="compression<br>ratio",
    margin=dict(l=10, r=10, t=10, b=10),
    font=dict(
        family="Courier New, monospace",
        size=25,
    ),
    bargap=0.10,
    bargroupgap=0.0,
)
new = q.bench_num
new[-1] = 'gmean'
fig.update_xaxes(tickangle=-45,
    type='category',
    tickvals=np.arange(len(new)),
    ticktext=new,
    linewidth=0,
)

fig.write_image("img/constrained_geo_micro.pdf", 
                width=2*w*dpi+20,
                height=h*dpi+20)

### Norm

In [12]:
fig = go.Figure()
fig.update_layout(
    # template='simple_white',
    title=f"Norm XOR boost snapshot profiling (constrained,geomean)\
        <br><sup>BDI and dedup</sup>",
        yaxis_title="Norm. compression ratio",
        xaxis_title=None,
    font=dict(
        family="Courier New, monospace",
        size=18,
    ),
    legend=dict(
        orientation="h",
        itemwidth=30,
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5,
    ),
    barmode='group',
)
# fig.update_yaxes(range = [0,4])

i = 0
colors = [px.colors.sequential.Purp,px.colors.sequential.Tealgrn]
hatch = ['', '-']
for comp in df_c_intra.compression.unique():
    for xor in df_c_intra.xor.unique():
        q_overall = df_c_intra.loc[(df_c_intra['compression']==comp) & (df_c_intra['xor']==xor)]
        q_overall_both = df_c_both.loc[(df_c_both['compression']==comp) & (df_c_both['xor']==xor)]
        q_norm = q_overall['bench_norm'].values[0]
        q_both_norm = q_overall_both['bench_norm'].values[0]
        color = colors[int(q_overall.compression=='bdi')][i]
        color_both = colors[int(q_overall_both.compression=='bdi')][i]

        fig.add_trace(
            go.Bar(
                x=q_both_norm.index, 
                y=q_both_norm.cr_geo, 
                name=f'{comp}_{xor} both', 
                opacity=0.5,
                offsetgroup=i,
                marker_color=color,
                showlegend=False,
                marker_pattern_shape=[hatch[x]for x in q_both_norm.category=='float']
            ),
        )

        fig.add_trace(
            go.Bar(
                x=q_norm.index, 
                y=q_norm.cr_geo, 
                name=f'{comp}_{xor}', 
                opacity=1,
                offsetgroup=i,
                marker_color=color_both,
                marker_pattern_shape=[hatch[x]for x in q_norm.category=='float']
            ),
        )

        i+=1

fig.show()

# write static image
dpi = 300
fig.update_traces(marker_line_width = 0,
                  selector=dict(type="bar"),
                  marker_pattern_shape=None,)

fig.update_layout(
    template='plotly_white',
    width=2*w*dpi,# double col
    height=h*dpi,
    paper_bgcolor='rgba(0,0,0,0)',
    title=None,
    xaxis_title=None,
    yaxis_title="Norm. compression<br>ratio",
    margin=dict(l=10, r=10, t=10, b=10),
    font=dict(
        family="Courier New, monospace",
        size=25,
    ),
    bargap=0.10,
    bargroupgap=0.0,
)
new = q.bench_num
new[-1] = 'gmean'
fig.update_xaxes(tickangle=-45,
    type='category',
    tickvals=np.arange(len(new)),
    ticktext=new,
    linewidth=0,
)

fig.write_image("img/constrained_geo_norm_micro.pdf", 
                width=2*w*dpi+20,
                height=h*dpi+20)

check all snapshots profiled

In [13]:
fig_ss = go.Figure()
fig_ss.update_layout(
    title=f"XOR boost snapshot profiling\
        <br><sup>BDI and dedup</sup>",
        yaxis_title="# snapshots",
        xaxis_title="combinations",
    font=dict(
        family="Courier New, monospace",
        size=18,
    )
)
i = 0
colors = [px.colors.sequential.Purp,px.colors.sequential.Tealgrn]
for comp in df_c_intra.compression.unique():
    for xor in df_c_intra.xor.unique():
        q_overall = df_c_intra.loc[(df_c_intra['compression']==comp) & (df_c_intra['xor']==xor)]
        q_overall_both = df_c_both.loc[(df_c_both['compression']==comp) & (df_c_both['xor']==xor)]
        q = q_overall['bench'].values[0]
        q_both = q_overall_both['bench'].values[0]
        color = colors[int(q_overall.compression=='bdi')][i]
        color_both = colors[int(q_overall_both.compression=='bdi')][i]

        fig_ss.add_trace(
            go.Bar(
                x=q_both.index, 
                y=q_both.ss_actual, 
                name=f'{comp}_{xor} both', 
                opacity=1,
                marker_color=color,
                error_y=dict(
                    type='data',
                    array=q_both.ss_theoretical-q_both.ss_actual,
                    arrayminus=q_both.ss_theoretical - q_both.ss_theoretical,
                    visible=True,
                ),
                showlegend=False,
            ),
        )

        fig_ss.add_trace(
            go.Bar(
                x=q.index, 
                y=q.ss_actual, 
                name=f'{comp}_{xor}', 
                opacity=1,
                marker_color=color_both,
                marker_pattern_shape="x",
                error_y=dict(
                    type='data',
                    array=q.ss_theoretical-q.ss_actual,
                    arrayminus=q.ss_theoretical - q.ss_theoretical,
                    visible=True,
                )
            ),
        )

        i+=1

# fig_ss.show()

# unconstrained profiling
### any blk in L2 can be xored with another in L2

narrow down - unconstrained

In [14]:
df_uc_intra = df.query("config == 'unconstrained' and slice == 'intra-only'")
display(df_uc_intra)

df_uc_both = df.query("config == 'unconstrained' and slice == 'both-inter-intra'")
display(df_uc_both)

,compression,xor,config,slice,bench,bench_norm
18,bdi,none,unconstrained,intra-only,ss_theoretical bench_num category ...,ss_theoretical bench_num category c...
19,bdi,rand,unconstrained,intra-only,ss_theoretical bench_num category ...,ss_theoretical bench_num category ...
20,bdi,ideal,unconstrained,intra-only,ss_theoretical bench_num category ...,ss_theoretical bench_num category ...
21,dedup,none,unconstrained,intra-only,ss_theoretical bench_num category ...,ss_theoretical bench_num category c...
22,dedup,rand,unconstrained,intra-only,ss_theoretical bench_num category ...,ss_theoretical bench_num category c...
23,dedup,ideal,unconstrained,intra-only,ss_theoretical bench_num category ...,ss_theoretical bench_num category c...


,compression,xor,config,slice,bench,bench_norm
6,bdi,none,unconstrained,both-inter-intra,ss_theoretical bench_num category ...,ss_theoretical bench_num category c...
7,bdi,rand,unconstrained,both-inter-intra,ss_theoretical bench_num category ...,ss_theoretical bench_num category ...
8,bdi,ideal,unconstrained,both-inter-intra,ss_theoretical bench_num category ...,ss_theoretical bench_num category ...
9,dedup,none,unconstrained,both-inter-intra,ss_theoretical bench_num category ...,ss_theoretical bench_num category c...
10,dedup,rand,unconstrained,both-inter-intra,ss_theoretical bench_num category ...,ss_theoretical bench_num category c...
11,dedup,ideal,unconstrained,both-inter-intra,ss_theoretical bench_num category ...,ss_theoretical bench_num category c...


### Abs

In [15]:
fig = go.Figure()
fig.update_layout(
    # template='simple_white',
    title=f"XOR boost snapshot profiling (unconstrained,geomean)\
        <br><sup>BDI and dedup</sup>",
        yaxis_title="compression ratio",
        xaxis_title=None,
    font=dict(
        family="Courier New, monospace",
        size=18,
    ),
    legend=dict(
        orientation="h",
        itemwidth=30,
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5,
    ),
    barmode='group',
)
# fig.update_yaxes(range = [0,4])

i = 0
colors = [px.colors.sequential.Purpor,px.colors.sequential.Blugrn]
for comp in df_uc_intra.compression.unique():
    for xor in df_uc_intra.xor.unique():
        q_overall = df_uc_intra.loc[(df_uc_intra['compression']==comp) & (df_uc_intra['xor']==xor)]
        q_overall_both = df_uc_both.loc[(df_uc_both['compression']==comp) & (df_uc_both['xor']==xor)]
        q = q_overall['bench'].values[0]
        q_both = q_overall_both['bench'].values[0]
        color = colors[int(q_overall.compression=='bdi')][i]
        color_both = colors[int(q_overall_both.compression=='bdi')][i]

        fig.add_trace(
            go.Bar(
                x=q_both.index, 
                y=q_both.cr_geo, 
                name=f'{comp}_{xor} both', 
                opacity=0.5,
                offsetgroup=i,
                marker_color=color,
                showlegend=False,
                marker_pattern_shape=[hatch[x]for x in q_both.category=='float']
            ),
        )

        fig.add_trace(
            go.Bar(
                x=q.index, 
                y=q.cr_geo, 
                name=f'{comp}_{xor}', 
                opacity=1,
                offsetgroup=i,
                marker_color=color_both,
                marker_pattern_shape=[hatch[x]for x in q.category=='float']
            ),
        )

        i+=1

fig.show()

# write static image
dpi = 300
fig.update_traces(marker_line_width = 0,
                  selector=dict(type="bar"),
                  marker_pattern_shape=None,)

fig.update_layout(
    template='plotly_white',
    width=2*w*dpi,# double col
    height=h*dpi,
    paper_bgcolor='rgba(0,0,0,0)',
    title=None,
    xaxis_title=None,
    yaxis_title="compression<br>ratio",
    margin=dict(l=10, r=10, t=10, b=10),
    font=dict(
        family="Courier New, monospace",
        size=25,
    ),
    bargap=0.10,
    bargroupgap=0.0,
)
new = q.bench_num
new[-1] = 'gmean'
fig.update_xaxes(tickangle=-45,
    type='category',
    tickvals=np.arange(len(new)),
    ticktext=new,
    linewidth=0,
)
fig.update_yaxes(
    gridwidth=0
)
# fig.for_each_trace(lambda t: t.update(marker_))

fig.write_image("img/unconstrained_geo_micro.pdf", 
                width=2*w*dpi+50,
                height=h*dpi+50)

### Abs

In [16]:
fig = go.Figure()
fig.update_layout(
    # template='simple_white',
    title=f"Norm XOR boost snapshot profiling (unconstrained,geomean)\
        <br><sup>BDI and dedup</sup>",
        yaxis_title="Norm. compression ratio",
        xaxis_title=None,
    font=dict(
        family="Courier New, monospace",
        size=18,
    ),
    legend=dict(
        orientation="h",
        itemwidth=30,
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5,
    ),
    barmode='group',
)
# fig.update_yaxes(range = [0,4])

i = 0
hatch = ['', '-']
for comp in df_uc_intra.compression.unique():
    for xor in df_c_intra.xor.unique():
        q_overall = df_uc_intra.loc[(df_uc_intra['compression']==comp) & (df_uc_intra['xor']==xor)]
        q_overall_both = df_uc_both.loc[(df_uc_both['compression']==comp) & (df_uc_both['xor']==xor)]
        q_norm = q_overall['bench_norm'].values[0]
        q_both_norm = q_overall_both['bench_norm'].values[0]
        color = colors[int(q_overall.compression=='bdi')][i]
        color_both = colors[int(q_overall_both.compression=='bdi')][i]

        fig.add_trace(
            go.Bar(
                x=q_both_norm.index, 
                y=q_both_norm.cr_geo, 
                name=f'{comp}_{xor} both', 
                opacity=0.5,
                offsetgroup=i,
                marker_color=color,
                showlegend=False,
                marker_pattern_shape=[hatch[x]for x in q_both_norm.category=='float']
            ),
        )

        fig.add_trace(
            go.Bar(
                x=q_norm.index, 
                y=q_norm.cr_geo, 
                name=f'{comp}_{xor}', 
                opacity=1,
                offsetgroup=i,
                marker_color=color_both,
                marker_pattern_shape=[hatch[x]for x in q_norm.category=='float']
            ),
        )

        i+=1

fig.show()

# write static image
dpi = 300
fig.update_traces(marker_line_width = 0,
                  selector=dict(type="bar"),
                  marker_pattern_shape=None,)

fig.update_layout(
    template='plotly_white',
    width=2*w*dpi,# double col
    height=h*dpi,
    paper_bgcolor='rgba(0,0,0,0)',
    title=None,
    xaxis_title=None,
    yaxis_title="Norm. compression<br>ratio",
    margin=dict(l=10, r=10, t=10, b=10),
    font=dict(
        family="Courier New, monospace",
        size=25,
    ),
    bargap=0.10,
    bargroupgap=0.0,
)
new = q.bench_num
new[-1] = 'gmean'
fig.update_xaxes(tickangle=-45,
    type='category',
    tickvals=np.arange(len(new)),
    ticktext=new,
    linewidth=0,
)

fig.write_image("img/unconstrained_geo_norm_micro.pdf", 
                width=2*w*dpi+20,
                height=h*dpi+20)

check all snapshots profiled

In [17]:
i = 0
for comp in df_uc_intra.compression.unique():
    for xor in df_uc_intra.xor.unique():
        q_overall = df_uc_intra.loc[(df_uc_intra['compression']==comp) & (df_uc_intra['xor']==xor)]
        q_overall_both = df_uc_both.loc[(df_uc_both['compression']==comp) & (df_uc_both['xor']==xor)]
        q = q_overall['bench'].values[0]
        q_both = q_overall_both['bench'].values[0]
        color = colors[int(q_overall.compression=='bdi')][i]
        color_both = colors[int(q_overall_both.compression=='bdi')][i]

        fig_ss.add_trace(
            go.Bar(
                x=q_both.index, 
                y=q_both.ss_actual, 
                name=f'{comp}uc_{xor} both', 
                opacity=1,
                marker_color=color,
                error_y=dict(
                    type='data',
                    array=q_both.ss_theoretical-q_both.ss_actual,
                    arrayminus=q_both.ss_theoretical - q_both.ss_theoretical,
                    visible=True,
                ),
                showlegend=False,
            ),
        )

        fig_ss.add_trace(
            go.Bar(
                x=q.index, 
                y=q.ss_actual, 
                name=f'{comp}uc_{xor}', 
                opacity=1,
                marker_color=color_both,
                marker_pattern_shape="x",
                error_y=dict(
                    type='data',
                    array=q.ss_theoretical-q.ss_actual,
                    arrayminus=q.ss_theoretical - q.ss_theoretical,
                    visible=True,
                )
            ),
        )

        i+=1

# fig_ss.show()